In [1]:
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
import pendulum
import operator as op

driver = uc.Chrome()
time.sleep(3)
driver.get('https://www.indeed.com/q-usa-jobs.html')
time.sleep(6)

In [2]:
keyword_filter= "chief"
location_filter = "california"
page_count_filter = "3"
salary_filter = "120000 $ hour"
start_date_posted_filter = "2024-07-01 00:00:00"
end_date_posted_filter = "2025-01-31 23:59:59"
time.sleep(3)

In [3]:
search_job_keyword = driver.find_element(By.XPATH, '//input[@aria-label="search: Job title, keywords, or company"]')
time.sleep(3)
search_job_keyword.click()
time.sleep(3)
actions = ActionChains(driver)
time.sleep(3)
actions.key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL).send_keys(Keys.BACK_SPACE).perform()
time.sleep(3)
search_job_keyword.send_keys(keyword_filter)
time.sleep(3)
search_location_keyword = driver.find_element(By.XPATH, '//input[@aria-label="Edit location"]')
time.sleep(3)
search_location_keyword.click()
time.sleep(3)
search_location_keyword.send_keys(location_filter)
time.sleep(3)
search_button = driver.find_element(By.XPATH, '//button[@class="yosegi-InlineWhatWhere-primaryButton"]')
time.sleep(3)
search_button.submit()
time.sleep(6)

In [4]:
def search_job_indeed():
    global search_job_indeed_list, search_job_indeed_final_list, search_job_indeed_final_list_filter
    search_job_indeed_final_list = []
    time.sleep(3)
    while True:
        try:
            job_indeed_frame = driver.find_elements(By.XPATH, '//div[@id="mosaic-provider-jobcards"]/ul/li/div/div/div/div/div/div/table/tbody/tr/td[1]/div[1]/h2/a')
            time.sleep(3)
            search_job_indeed_list = list(set((job_indeed.get_attribute('href') for job_indeed in job_indeed_frame)))
            time.sleep(3)
            job_indeed_frame[-1].location_once_scrolled_into_view
            time.sleep(6)
        except:
            print('No more job')
            time.sleep(3)
        search_job_indeed_final_list.extend(search_job_indeed_list)
        time.sleep(3)
        search_job_indeed_final_list = list(set(search_job_indeed_final_list))
        time.sleep(3)
        if (len(search_job_indeed_final_list) % 15 == 0 and len(search_job_indeed_final_list) // 15 == int(page_count_filter)) or (len(search_job_indeed_final_list) % 15 != 0 and len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter)):
            break
        try:
            next_page_button = driver.find_elements(By.XPATH, '//nav[@aria-label="pagination"]/ul/li/a')
            time.sleep(3)
            next_page_button[-1].click()
            time.sleep(3)
            yield len(search_job_indeed_final_list)
        except:
            print('No more page')
            time.sleep(3)
            break

generators = search_job_indeed()
for generator in generators:
    print(generator)

15
30


In [5]:
if len(search_job_indeed_final_list) % 15 == 0:
    if len(search_job_indeed_final_list) == 0:
        search_job_indeed_final_list_filter = []
    elif len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]
else:
    if len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    elif len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:(int(page_count_filter) - 1) * 15 + len(search_job_indeed_final_list) % 15]
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]

In [6]:
len(search_job_indeed_final_list_filter)

45

In [ ]:
def get_job_indeed_description():
        global job_indeed_description_list
        job_indeed_description_list = []
        for _, job_indeed_url in enumerate(search_job_indeed_final_list_filter):
            driver.get(job_indeed_url)
            time.sleep(6)
            try:
                    company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                    time.sleep(3)
                    sub_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[2]/span').text
                    time.sleep(3)
                    if len(re.findall(r'\b\d+\.\d+\b', sub_company)) > 0:
                        total_company = company
                    else:
                        total_company = company + ' ' + sub_company
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                            time.sleep(3)
                    except:
                            try:
                                    time.sleep(3)
                                    total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span').text
                                    time.sleep(3)
                            except:
                                    try:
                                            time.sleep(3)
                                            total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                            time.sleep(3)
                                    except:
                                            try:
                                                    time.sleep(3)
                                                    total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                                    time.sleep(3)
                                            except:
                                                    time.sleep(3)
                                                    total_company = ''
                                                    time.sleep(3)
            try:
                    header = driver.find_element(By.XPATH, '//div[starts-with(@class,"jobsearch-JobInfoHeader-title-container")]/h1/span').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            header = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/span').text
                            time.sleep(3)
                    except:
                            time.sleep(3)
                            header = ''
                            time.sleep(3)
            try:
                    location = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div[2]/div').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/div[2]/div').text
                            time.sleep(3)
                    except:
                            try:
                                time.sleep(3)
                                location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div[2]/div/div/div/div[2]/div').text
                                time.sleep(3)
                            except:
                                    time.sleep(3)
                                    location = ''
                                    time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_context = soup.find_all('script', string=re.compile(r'@context'))[0].getText().strip()
                time.sleep(3)
                standard_date_posted = re.search(r'\"datePosted\":\"([^"]+)\"', script_context).group(1)
                time.sleep(3)
                readable_date_posted = pendulum.parse(standard_date_posted, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                time.sleep(3)
            except:
                try:
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    time.sleep(3)
                    script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                    time.sleep(3)
                    standard_date_posted = re.search(r'\"datePublished\":(\d+)', script_initial_data).group(1)
                    time.sleep(3)
                    readable_date_posted = pendulum.from_timestamp(int(standard_date_posted) / 1000, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                    time.sleep(3)
                except:
                        time.sleep(3)
                        readable_date_posted = ''
                        time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                time.sleep(3)
                salary = re.search(r'\"salaryText\":\"([^"]+)\"', script_initial_data).group(1)
                time.sleep(3)
            except:
                    time.sleep(3)
                    salary = ''
                    time.sleep(3)
            job_indeed_description_list.append((job_indeed_url, total_company, header, salary, location, readable_date_posted))
            yield total_company, header, location, readable_date_posted, salary
generators = get_job_indeed_description()
for generator in generators:
    print(generator)

('LA Philharmonic', 'Chief Philanthropy Officer', 'Los Angeles, CA', '2024-12-02 22:58:01', '$400,000 - $450,000 a year')


In [23]:
len(job_indeed_description_list)

14

In [24]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
import pandas as pd

credentials = service_account.Credentials.from_service_account_file('credentials.json')
spreadsheet_service = build('sheets', 'v4', credentials=credentials)

SPREADSHEET_ID = '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg'
RANGE_NAME = 'Sheet1'
sheet = spreadsheet_service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])
max_cols = max(len(row) for row in values)
values = [row + [''] * (max_cols - len(row)) for row in values]
df = pd.DataFrame(values[1:], columns=values[0])
df = df.fillna('')

In [25]:
cleared_values = {
    "requests": [
        {
            "updateCells": {
                "range": {
                    "sheetId": 0,
                    "startRowIndex": 1,
                    "endRowIndex": None,
                    "startColumnIndex": 0,
                    "endColumnIndex": 6
                },
                "fields": "userEnteredValue"
            }
        }
    ]
}
spreadsheet_service.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID, body=cleared_values).execute()

{'spreadsheetId': '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg',
 'replies': [{}]}

In [26]:
df1 = pd.DataFrame(job_indeed_description_list, columns=df.columns).__deepcopy__()
df1

,Link,Company,Title,Salary,Location,Date posted
0,https://www.indeed.com/rc/clk?jk=88cea759d73cc...,Career Technologies USA,Material Handler,$18 - $20 an hour,"Chatsworth, CA",2024-10-21 22:24:25
1,https://www.indeed.com/rc/clk?jk=2edda3887ee30...,AGQ Labs USA,Laboratory Analyst,"$69,000 - $76,000 a year","AGQ Labs USA in Oxnard, CA 93030",2024-12-23 17:35:59
2,https://www.indeed.com/rc/clk?jk=0d134161edf6b...,GOBankingRates,New Business Development Specialist (100% Remo...,"$80,000 - $100,000 a year","Los Angeles, CA",2025-01-08 00:37:58
3,https://www.indeed.com/rc/clk?jk=a8c2ba87d1c56...,TePe USA,Call Center Agent,,"1130 North Gilbert Street, Anaheim, CA 92801",2023-01-09 16:56:12
4,https://www.indeed.com/rc/clk?jk=bc79229d78a77...,Wisetek,Driver,,"Sacramento, CA",2023-12-30 02:20:51
5,https://www.indeed.com/rc/clk?jk=6aece1b4aaadd...,Domino's Corporate,"Production Associate (California, USA)",$22 an hour,"301 South Rockefeller Avenue, Ontario, CA 91764",2025-01-09 19:49:13
6,https://www.indeed.com/rc/clk?jk=f6a7501fd9751...,Vertogic,Software Engineers,,"Fremont, CA",2020-11-09 15:59:40
7,https://www.indeed.com/rc/clk?jk=a90fcf7b58e83...,Advantech USA,Shipping and Receiving Clerk,$20 - $22 an hour,"Milpitas, CA",2025-01-09 00:24:01
8,https://www.indeed.com/rc/clk?jk=274a838921878...,Reliant Holdings,Processing - Ice Operator Visalia California,$18 - $22 an hour,"7249 W Goshen Ave, Visalia, CA 93291",2024-03-20 22:29:30
9,https://www.indeed.com/rc/clk?jk=a30eaa1113bec...,Lumentum,FinFet Mask Design,,"1001 Ridder Park Dr, San Jose, CA 95131",2023-01-18 01:22:01


In [27]:
def implement_salary_filter():
    global implement_filter_list
    implement_filter_list = []
    salary_filter_tuple = (salary_filter.split(' ')[0], salary_filter.split(' ')[1], salary_filter.split(' ')[2])
    for index, row in df1.iterrows():
        if op.contains(df1.at[index, 'Salary'], salary_filter_tuple[1]) and op.contains(df1.at[index, 'Salary'], salary_filter_tuple[2]):
            range_salary_tuple = tuple(int(salary.replace(',', '')) for salary in re.findall(r"([\d,]+)", df1.iloc[index]['Salary']))
            print(range_salary_tuple)
            if len(range_salary_tuple) == 2 and (int(salary_filter_tuple[0]) >= range_salary_tuple[0] and int(salary_filter_tuple[0]) <= range_salary_tuple[1]):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
                yield from implement_filter_list
            elif len(range_salary_tuple) == 1 and (int(salary_filter_tuple[0]) <= range_salary_tuple[0] and op.contains(df1.iloc[index]['Salary'].lower(), 'up to')):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
                yield from implement_filter_list
            elif len(range_salary_tuple) == 1 and (int(salary_filter_tuple[0]) >= range_salary_tuple[0] and op.contains(df1.iloc[index]['Salary'].lower(), 'from')):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
                yield from implement_filter_list
            elif len(range_salary_tuple) == 1 and (int(salary_filter_tuple[0]) == range_salary_tuple[0] and (not op.contains(df1.iloc[index]['Salary'].lower(), 'up to') and not op.contains(df1.iloc[index]['Salary'].lower(), 'from'))):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
                yield from implement_filter_list
salary_generators = implement_salary_filter()
for generator in salary_generators:
    print(generator)

(18, 20)
(22,)
(20, 22)
(18, 22)
(20, 22)
(18, 0, 29, 86)


In [28]:
df2 = pd.DataFrame(implement_filter_list, columns=df1.columns).__deepcopy__()
implement_filter_list = []

In [29]:
def implement_date_posted_filter():
    for row in df2.itertuples(index=False):
        if pendulum.parse(start_date_posted_filter, tz='Asia/Ho_Chi_Minh') <= pendulum.parse(row._5, tz='Asia/Ho_Chi_Minh') and pendulum.parse(end_date_posted_filter, tz='Asia/Ho_Chi_Minh') >= pendulum.parse(row._5, tz='Asia/Ho_Chi_Minh'):
            implement_filter_list.append((row.Link, row.Company, row.Title, row.Salary, row.Location, row._5))
            yield from implement_filter_list
date_posted_generators = implement_date_posted_filter()
for generator in date_posted_generators:
    print(generator)
implement_filter_list = list(set(implement_filter_list))

In [30]:
df3 = pd.DataFrame(implement_filter_list, columns=df1.columns).__deepcopy__()
updated_values = [df3.columns.tolist()] + df3.values.tolist()
body = {'values': updated_values}
spreadsheet_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME,
    valueInputOption='RAW', body=body).execute()

{'spreadsheetId': '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg',
 'updatedRange': 'Sheet1!A1:F1',
 'updatedRows': 1,
 'updatedColumns': 6,
 'updatedCells': 6}